In [11]:
import pandas as pd
import numpy as np
import json
from together import Together
import os
from dotenv import load_dotenv
from together import Together

os.chdir("C:\\Users\\siyak\\NLP-Final-Project")


In [1]:
API_KEY = "69345dcf4dfc4e5222bb99aabeb24904e74ff20e4ea70c83cd89422ef8620bd8"  
client = Together(api_key=API_KEY)

import csv

high_engagement_posts = []

with open("selected_posts_with_physician_comments.csv", "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)  # Read as CSV
    for row in reader:
        if int(row.get("score", 0)) > 0:  # Ensure score is numeric
            high_engagement_posts.append(row)

if not high_engagement_posts:
    print("No high-engagement posts found.")
    exit()

NameError: name 'Together' is not defined

In [13]:
high_engagement_posts.sort(key=lambda x: x.get("score", 0), reverse=True)

# Select top 50% of posts to get posts with the most engagement
top_half_index = len(high_engagement_posts) // 2
top_engagement_posts = high_engagement_posts[:top_half_index]

# Only process 100 posts at first
max_posts = min(len(top_engagement_posts), 100)
top_engagement_posts = top_engagement_posts[:max_posts]

In [14]:
df_selected_posts = pd.DataFrame(top_engagement_posts)
df_selected_posts.to_csv("selected_posts_with_comments.csv", index=False)

In [15]:
results_prompt1 = []
results_prompt2 = []

for post in top_engagement_posts:
    post_id = post.get("id", "No id")
    title = post.get("title", "No title")
    body = post.get("selftext", "No content available")
    score = post.get("score", 0)

    # Prompt 1: Naive Approach - Medical Diagnosis Suggestion
    prompt1 = (
        f"Based on the following symptoms, list possible medical conditions and recommend "
        f"the next diagnostic tests or evaluations a doctor might order. "
        f"Symptoms: {body}. Include brief justifications for each recommendation."
    )

    response1 = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt1}],
    )

    # Prompt 2: Reddit-Style Physician Response
    prompt2 = (
        f"You're replying to a Reddit post in r/AskDocs. The post asks: {body}. "
        f"Write a comment like a real Reddit user who is a verified physician."
    )

    response2 = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt2}],
    )

    common_data = {"id": id, "title": title, "body": body, "score": score}

    results_prompt1.append({**common_data, "generated_response": response1.choices[0].message.content})
    results_prompt2.append({**common_data, "generated_response": response2.choices[0].message.content})

df_prompt1 = pd.DataFrame(results_prompt1)
df_prompt2 = pd.DataFrame(results_prompt2)

df_prompt1.to_csv("prompt1_responses_with_physician_comments.csv", index=False)
df_prompt2.to_csv("prompt2_responses_with_physician_comments.csv", index=False)

print(f"Processed {len(results_prompt1)} posts. Results saved to 'prompt1_responses_with_physician_comments.csv' and 'prompt2_responses_with_physician_comments.csv'.")

Processed 47 posts. Results saved to 'prompt1_responses_with_physician_comments.csv.csv' and 'prompt2_responses_with_physician_comments.csv.csv'.
